In [1]:
import datetime as dt
import logging as log

%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb
%run Metric.ipynb
%run Operation.ipynb
%run Account.ipynb
%run Policy_Loader.ipynb
%run Evaluation.ipynb


class RocketRegression:
    
    def __init__(self, policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        
    def start_regression_on_one_stock(self,symbol,regression_end_date):
        sectionManager = BreakSectionManager(symbol,self.context)
        section_df = sectionManager.get_all_break_sections()
        section_df = section_df[section_df['d_s_datetime']<to_datetime(regression_end_date)]
        
        metric = Metric(symbol,self.context)
        
        stock_start_date = metric.get_stock_start_date()
        stock_end_date = metric.get_stock_end_date()
        log.info('stock_start_date ' + stock_start_date+' stock_end_date:'+stock_end_date)
        
        account = Account(stock_start_date,stock_end_date,self.context,self.policy_name)
        operation = Operation(account,symbol,self.context,self.policy_name)
        
        for key,row in section_df.iterrows():
            start_date = row['d_s_date']
            end_date = row['d_e_date']
            
            start_datetime = to_datetime(start_date)
            end_datetime = to_datetime(end_date)

            buy_price = metric.get_cur_price(start_date)
            
            if(self.debug==True):
                print('\nsection '+start_date+'===>'+end_date)
            while(start_datetime<=end_datetime):
                cur_date = start_datetime.strftime(YMD_format)
                
                if(metric.is_today_open(cur_date)):
                    account.daily_audit(cur_date)

                    if(account.can_open_new_stock()==True):
                        if(operation.is_buy_point(cur_date)==True):
                            operation.open_opsition(cur_date)

                    if(account.has_shares()==True):
                        if(operation.is_sell_point(cur_date)==True):
                            operation.sell_stock(cur_date)

                start_datetime += dt.timedelta(days = 1)
                    
                        
        operation_df = operation.get_operations()
        
        evaluation = Evaluation(account,self.context)
        evaluation_df = evaluation.get_evaluation_report(symbol,stock_start_date,stock_end_date,
                                                        self.policy_name,operation_df)

        return evaluation_df
    
    
    def start_regression(self,scale,regression_end_date):
        print('context:' + str(self.context))
        
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        
        symbols = get_symbols(scale)
        
        progress = 0
        total = len(symbols) 
        for symbol in symbols:
            one_evaluation = self.start_regression_on_one_stock(symbol,regression_end_date)
            
            evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
            
            progress += 1
            if(progress%4==0):
                print(round(100.0*progress/total,2),end=' ')
                
        
        return evaluation_df
    

In [2]:
regression = RocketRegression('rocket_break_baseline.json')

regression_end='2019-05-30'
scale=10
evaluation_df = regression.start_regression(scale,regression_end)
evaluation_df

context:{'short': 5, 'mid': 10, 'scale_start': 0, 'scale_end': 3, 'level': 'Day'}

year_returns
[-23.42, -23.83, 8.98, -33.98, -8.23, 6.21, 2.93, 57.57, -49.26, 37.53, -34.21, -15.07, 1.12, 17.78, -8.44, 35.07, 5.13, -9.75, -21.39]

year_returns
[37.99, -30.19, -26.55, -13.17, -9.71, -11.23, -12.99, 56.67, -43.53, -0.77, -25.15, -80.64, 4.98, 13.0, 6.34, 64.11, -13.96, -27.74, -43.34, -10.9]

year_returns
[-4.39, 39.07, -13.78, 47.8, -124.76, 32.21, 23.25, -16.35, -36.7, 13.49, -15.58, 21.39, -37.29, -9.09, -15.11, -5.68]

year_returns
[6.52, -57.67, 18.61, 12.8, 11.99, -8.82, -25.01, 33.39, 32.17, -42.7, -8.4, -4.68, -0.23]

year_returns
[-27.3, -79.7, -16.93, -20.94, 38.54, 54.85, -36.13, -29.0, -44.3, -9.82]
build SZ#002504 break sections
start to analyse Day level SZ#002504 all break sections...

year_returns
[-41.63, -36.7, 7.41, 11.41, 63.98, -47.7, -4.57, 32.75, -5.29]

year_returns
[-31.72, -13.9, 11.04, 8.92, -7.44, -27.25, 53.63, 61.63, -56.05, 44.6, -9.24, -9.36, 16.18, 7.62

,symbol,start_date,end_date,policy_name,short,mid,max_drawdown,loss_times,year_return,mean_year_return,std_year_return,mean_month_return,std_month_return
0,SZ#002021,2004-07-15,2019-02-18,rocket_break_baseline.json,5,10,69.10,5,27.91,-0.395333,32.619616,0.618750,11.592944
1,SZ#000806,2000-02-14,2019-02-18,rocket_break_baseline.json,5,10,61.84,6,57.07,-0.237500,30.987112,0.971500,11.340826
2,SZ#000576,2000-02-14,2019-02-18,rocket_break_baseline.json,5,10,51.71,5,447.40,10.688000,24.017460,2.009939,10.514571
3,SZ#000001,2000-02-14,2019-02-18,rocket_break_baseline.json,5,10,55.14,6,106.67,3.843000,28.023225,1.451203,10.314183
4,SZ#002504,2010-11-18,2019-02-18,rocket_break_baseline.json,5,10,53.26,4,23.89,-2.260000,36.587629,0.902923,11.924351
5,SZ#002343,2010-01-26,2019-02-18,rocket_break_baseline.json,5,10,74.06,13,-30.57,-17.073000,38.882685,-0.701948,11.583921
6,SZ#002181,2007-11-16,2019-02-18,rocket_break_baseline.json,5,10,59.11,5,0.44,-2.463846,26.944779,0.653173,12.041591
7,SZ#002020,2004-07-15,2019-02-18,rocket_break_baseline.json,5,10,70.54,8,3.29,-6.345000,40.619891,0.132222,12.718396
8,SZ#000803,2000-02-14,2019-02-18,rocket_break_baseline.json,5,10,75.79,17,-6.35,-8.339000,33.856745,0.327765,11.684475
9,SZ#000573,2000-02-14,2019-02-18,rocket_break_baseline.json,5,10,57.00,8,4.48,-2.908421,26.925849,0.349939,9.718840


In [3]:
evaluation_df.describe()

,max_drawdown,year_return,mean_year_return,std_year_return,mean_month_return,std_month_return
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,62.755000,63.423000,-2.549110,31.946499,0.671547,11.343410
std,8.919131,140.211388,7.342096,5.561643,0.741228,0.902927
min,51.710000,-30.570000,-17.073000,24.017460,-0.701948,9.718840
25%,55.605000,1.152500,-5.485855,27.214390,0.333309,10.721135
50%,60.475000,14.185000,-2.361923,31.803364,0.635962,11.588433
75%,70.180000,49.780000,-0.276958,35.904908,0.954356,11.864382
max,75.790000,447.400000,10.688000,40.619891,2.009939,12.718396
